In [1]:
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline
import sys
from langchain_classification_utils import run_evaluation
import pandas as pd  
from src import comments 
from src import prompts
  
# The path you want to add  
path_to_add = r'/home/etaylor/code_projects/dubby'  
  
# Add the directory to sys.path  
sys.path.insert(0, path_to_add)

C:\Users\elorberb\AppData\Local\Temp\ipykernel_13456\123696342.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
# take only the first 100 rows
test_data_path = ""
df = pd.read_csv(test_data_path)
df = df[:100]

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1972 entries, 0 to 1971
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    1972 non-null   object
 1   label   1972 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 30.9+ KB


In [14]:
# Define the configurations for the experiments  
configurations = [  
    # Zero-shot  
    {  
        'template_type': 'zero_shot',  
        'template_text': prompts.template_zero_shot_only_comment,  
        'input_vars': ["comment", "start_task_context_prompt"],  
        'input_vars_values': {"start_task_context_prompt": prompts.start_task_context_prompt}  
    },  
    # Single-shot positive example  
    {  
        'template_type': 'single_shot_pos_example',  
        'template_text': prompts.template_single_shot_only_pos_comment,  
        'input_vars': ["comment", "start_task_context_prompt", "example_comment_pos"],  
        'input_vars_values': {"start_task_context_prompt": prompts.start_task_context_prompt,
                                "example_comment_pos": comments.comment_1_pos}
    },  
    # Single-shot negative example  
    {  
        'template_type': 'single_shot_neg_example',  
        'template_text': prompts.template_single_shot_only_neg_comment,  
        'input_vars': ["comment", "start_task_context_prompt", "example_comment_neg"],  
             'input_vars_values': {"start_task_context_prompt": prompts.start_task_context_prompt,
                                "example_comment_neg": comments.comment_1_neg}
    },  
    # Single-shot both examples  
    {  
        'template_type': 'single_shot_both_examples',  
        'template_text': prompts.template_single_shot_only_comment,  
        'input_vars': ["comment", "start_task_context_prompt", "example_comment_pos", "example_comment_neg"],  
        'input_vars_values': {"start_task_context_prompt": prompts.start_task_context_prompt,
                                "example_comment_pos": comments.comment_1_pos,
                                "example_comment_neg": comments.comment_1_neg}
    },  
    # Few-shot positive examples  
    {  
        'template_type': 'few_shot_pos_examples',  
        'template_text': prompts.template_few_shot_only_pos_comment,  
        'input_vars': ["comment", "start_task_context_prompt", "example_comment_1_pos", "example_comment_2_pos", 'example_comment_3_pos'],  
        'input_vars_values': {"start_task_context_prompt": prompts.start_task_context_prompt,
                                "example_comment_1_pos": comments.comment_1_pos,
                                "example_comment_2_pos": comments.comment_2_pos,
                                'example_comment_3_pos': comments.comment_3_pos}
    },  
    # Few-shot negative examples  
     {  
        'template_type': 'few_shot_neg_examples',  
        'template_text': prompts.template_few_shot_only_neg_comment,  
        'input_vars': ["comment", "start_task_context_prompt", "example_comment_1_neg", "example_comment_2_neg", 'example_comment_3_neg'],  
        'input_vars_values': {  
            "start_task_context_prompt": prompts.start_task_context_prompt,  
            "example_comment_1_neg": comments.comment_1_neg,  
            "example_comment_2_neg": comments.comment_2_neg,  
            'example_comment_3_neg': comments.comment_3_neg  
        }  
    },  
    # Few-shot both examples  
  {  
        'template_type': 'few_shot_examples',  
        'template_text': prompts.template_few_shot_only_comment,  
        'input_vars': ["comment", "start_task_context_prompt", "example_comment_1_pos", "example_comment_2_pos", 'example_comment_3_pos',
                       "example_comment_1_neg", "example_comment_2_neg", 'example_comment_3_neg'],  
        'input_vars_values': {
            "start_task_context_prompt": prompts.start_task_context_prompt,
            "example_comment_1_pos": comments.comment_1_pos,
            "example_comment_2_pos": comments.comment_2_pos,
            'example_comment_3_pos': comments.comment_3_pos,
            "example_comment_1_neg": comments.comment_1_neg,  
            "example_comment_2_neg": comments.comment_2_neg,  
            'example_comment_3_neg': comments.comment_3_neg
        }
  }
]


In [26]:
import json
# Assuming 'df' is the DataFrame with the data to evaluate  
num_records = 100  # Number of records to evaluate  
azure_deployment_name = 'gpt-35-turbo'  # Replace with your specific deployment name  
  
# Dictionary to store the results of all runs  
all_results = []  
  
# Iterate over all configurations and run the evaluation  
for config in configurations:  
    print(f"Running configuration: {config['template_type']}") 
    template_dict = {'template_type': config['template_type'], 'template_text': config['template_text']}
    result = run_evaluation(df, azure_deployment_name,
                            num_records,   
                            template_dict,   
                            config['input_vars'],   
                            config['input_vars_values'],  
                            verbose=True)  
    all_results.append(result)  
  
# Convert results to JSON and save to a file  
results_json = json.dumps(all_results, indent=4)
result_file_name = f'{azure_deployment_name}_evaluation_results.json'  
with open(result_file_name, 'w') as outfile:  
    outfile.write(results_json)  
  
print(f"All evaluations are complete and results are saved to {result_file_name}.")

Running configuration: zero_shot
Model initialized.


100it [00:23,  4.30it/s]


Predictions made.
Accuracy: 0.57
Precision: 0.6142207523770152
Recall: 0.57
Running configuration: single_shot_pos_example
Model initialized.


100it [00:21,  4.63it/s]


Predictions made.
Accuracy: 0.58
Precision: 0.7197368421052631
Recall: 0.58
Running configuration: single_shot_neg_example
Model initialized.


...

Predictions made.
Accuracy: 0.57
Precision: 0.7139469226425749
Recall: 0.57
Running configuration: few_shot_neg_examples
Model initialized.


100it [00:22,  4.47it/s]


Predictions made.
Accuracy: 0.67
Precision: 0.6686843211433375
Recall: 0.67
Running configuration: few_shot_neg_examples
Model initialized.


100it [00:22,  4.36it/s]

Predictions made.
Accuracy: 0.61
Precision: 0.6734448734448734
Recall: 0.61
All evaluations are complete and results are saved to gpt-35-turbo_evaluation_results.json.


In [18]:
print(len(all_results))

7
